In [ ]:
# Random Imports
import os
import sys
import yaml
from pathlib import Path
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSegDev')

# Regular schema dictates that we put DATAPATH
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
))
os.environ['WANDB_NOTEBOOK_NAME'] = 'calibrate.ipynb'

from ionpy.util import Config

# Setup direcrtories
root = Path("/storage/vbutoi/scratch/ESE")
scratch_root = Path("/storage/vbutoi/scratch/ESE")
code_root = Path("/storage/vbutoi/projects/ESE")

%load_ext yamlmagic
%load_ext autoreload
%autoreload 2

In [ ]:
%%yaml default_cfg

data:
  iters_per_epoch: 100 
  train_splits: cal
  val_splits: val 

dataloader:
  batch_size: 8 # Often, we train with a small num of images total. 
  num_workers: 1
  pin_memory: True 

optim: # Unclear if we should tune this or not.
  _class: torch.optim.Adam
  weight_decay: 0.0 
  lr: 1.0e-4

train:
  epochs: 300 # 10 * 100 = 1000 iterations
  eval_freq: 10 
  base_pretrained_dir: '?'
  base_checkpoint: 'max-val-dice_score'
  base_pt_select_metric: 'val-dice_score'

loss_func: 
  _class: '?'
  from_logits: True
  batch_reduction: 'mean' 

In [ ]:
%%yaml callbacks_cfg

callbacks:
  step:
    - ese.callbacks.ShowPredictions
  epoch:
    - ese.callbacks.WandbLogger
    - ionpy.callbacks.ETA
    - ionpy.callbacks.JobProgress
    - ionpy.callbacks.TerminateOnNaN
    - ionpy.callbacks.PrintLogged
    - ionpy.callbacks.ModelCheckpoint:
        monitor: 
          - ece_loss 
          - area_estimation_error
        phase: val

In [ ]:
%%yaml experiment_cfg 

name: "HepaticVessels_Cal_Pilot"

train: 
    base_pretrained_dir: 
        - "/storage/vbutoi/scratch/ESE/training/08_24_24_HeptaticVessel_LowerLR"

model:
    - TempScaling
    # - LTS 

data:
    _class: "ese.datasets.OCTA_6M"

loss_func:
    _class: 
        - ese.losses.SoftDiceLoss
        - ese.losses.PixelCELoss
        - ese.losses.AreaEstimationError

In [ ]:
from ese.analysis.analysis_utils.submit_utils import get_ese_calibration_configs

# Get the configs for the different runs.
base_cfg = Config(default_cfg).update([callbacks_cfg])

cal_cfgs = get_ese_calibration_configs(
    exp_cfg=experiment_cfg,
    base_cfg=base_cfg,
    calibration_model_cfgs={}
)

In [ ]:
len(cal_cfgs)

## Running Jobs

In [ ]:
from ese.experiment import run_ese_exp, submit_ese_exps, PostHocExperiment

In [ ]:
####### Run individual jobs
run_ese_exp(
    config=cal_cfgs[-1], 
    experiment_class=PostHocExperiment,
    run_name='debug',
    show_examples=True,
    track_wandb=False,
    gpu='0',
    # gpu='4',
)

In [ ]:
# #### Run Batch Jobs
# submit_ese_exps(
#     config_list=cal_cfgs,
#     experiment_class=PostHocExperiment,
#     track_wandb=True,
#     available_gpus=['0', '1', '2', '3']
#     # available_gpus=['4', '5', '6', '7']
# )